# Import de librerias

In [39]:
import http.client
import json
import pandas as pd

# Conexion con la API

In [40]:
conn = http.client.HTTPSConnection("idealista7.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "711a0e1079msh680a1c4987a2fe5p1621eejsnce62b684fc16",
    'x-rapidapi-host': "idealista7.p.rapidapi.com"
}

# Carga de datos

In [43]:
df_vivienda = pd.DataFrame()
df_localizacion = pd.DataFrame()

for i in range(1,40):
    conn.request("GET", f"/listhomes?order=relevance&operation=sale&locationId=0-EU-ES-28-07-001-079&locationName=Madrid&numPage={i}&maxItems=40&location=es&locale=en", headers=headers)

    res = conn.getresponse()
    json_data = res.read()

    if json_data:
        try:
            data_dict = json.loads(json_data)
        except json.JSONDecodeError as e:
            print("Error al decodificar JSON:", e)
    else:
        print("La respuesta está vacía.")
        break
        
    
    # Navegamos sobre el JSON
    if 'elementList' in data_dict:  # Reemplaza 'listings' con la clave correcta si es diferente
        data_list = data_dict['elementList']  # Accede a la lista de propiedades
    else:
        print("La clave 'listings' no se encuentra en el JSON.")
        data_list = []

    # Extraer solo los campos seleccionados y convertir en un DataFrame
    selected_fields = [
        {
            "propertyCode": item.get("propertyCode"),
            "url": item.get("url"),
            "address": item.get("address"),
            "size": item.get("size"),
            "floor": item.get("floor"),
            "province": item.get("province"),
            "municipality": item.get("municipality"),
            "district": item.get("district"),
            "price": item.get("price"),
            "rooms": item.get("rooms"),
            "hasLift": item.get("hasLift"),
            "hasParking": item.get("parkingSpace", {}).get("hasParkingSpace"),
            "hasTerrace": item.get("features", {}).get("hasTerrace"),
            "hasSwimmingPool": item.get("features", {}).get("hasSwimmingPool"),
            "hasAirConditioning": item.get("features", {}).get("hasAirConditioning"),
            "hasGarden": item.get("features", {}).get("hasGarden"),
            "bathrooms": item.get("bathrooms"),
            "latitude": item.get("latitude"),
            "longitude": item.get("longitude")
        }
        for item in data_list
    ]
    
    # Convertir la lista de diccionarios en un DataFrame
    df = pd.DataFrame(selected_fields)
    df_vivienda_aux = df.drop(['latitude', 'longitude'], axis=1)
    df_localizacion_aux = df[['propertyCode','latitude', 'longitude']]

    df_vivienda = pd.concat([df_vivienda,df_vivienda_aux], axis=0)
    df_localizacion = pd.concat([df_localizacion, df_localizacion_aux], axis=0)
        

Error al decodificar JSON: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 

In [44]:
csv_vivienda = df_vivienda.to_csv('../structured/vivienda.csv', index=False)
csv_localizacion = df_localizacion.to_csv('../structured/location.csv', index=False)

In [45]:
df_vivienda.describe()

,size,price,rooms,bathrooms
count,200.000000,2.000000e+02,200.00000,200.000000
mean,295.510000,1.908871e+06,3.87000,3.485000
std,290.961843,1.404916e+06,1.70223,1.878141
min,37.000000,1.749000e+05,0.00000,1.000000
25%,122.500000,8.980000e+05,3.00000,2.000000
50%,211.000000,1.472500e+06,4.00000,3.000000
75%,349.250000,2.552500e+06,5.00000,5.000000
max,1768.000000,6.890000e+06,9.00000,12.000000
